# Cartpole சமநிலையைப் பேண RL பயிற்சி

இந்த நோட்புக் [AI for Beginners Curriculum](http://aka.ms/ai-beginners) இன் ஒரு பகுதியாகும். இது [அதிகாரப்பூர்வ PyTorch டுடோரியல்](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) மற்றும் [இந்த Cartpole PyTorch செயல்பாடு](https://github.com/yc930401/Actor-Critic-pytorch) மூலம் ஊக்கமளிக்கப்பட்டது.

இந்த எடுத்துக்காட்டில், RL ஐப் பயன்படுத்தி ஒரு மாதிரியை ஒரு வண்டியில் ஒரு கம்பத்தை சமநிலைப்படுத்த பயிற்சி செய்யப் போகிறோம். அந்த வண்டி இடது மற்றும் வலது பக்கங்களில் நகர முடியும். [OpenAI Gym](https://www.gymlibrary.ml/) சூழலைப் பயன்படுத்தி கம்பத்தை சிமுலேட் செய்யப் போகிறோம்.

> **குறிப்பு**: இந்த பாடத்தின் குறியீட்டை உள்ளூரில் (எ.கா., Visual Studio Code மூலம்) இயக்கலாம், அப்போது சிமுலேஷன் ஒரு புதிய சாளரத்தில் திறக்கப்படும். குறியீட்டை ஆன்லைனில் இயக்கும்போது, [இங்கே](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) விவரிக்கப்பட்டுள்ளபடி, சில மாற்றங்களைச் செய்ய வேண்டியிருக்கும்.

முதலில் Gym நிறுவப்பட்டுள்ளதா என்பதை உறுதிப்படுத்துவோம்:


In [ ]:
import sys
!{sys.executable} -m pip install gym

இப்போது CartPole சூழலை உருவாக்கி அதில் செயல்படுவது எப்படி என்பதை பார்ப்போம். ஒரு சூழல் பின்வரும் பண்புகளை கொண்டுள்ளது:

* **Action space** என்பது ஒவ்வொரு சிமுலேஷன் படியில் நாம் செய்யக்கூடிய சாத்தியமான செயல்களின் தொகுப்பாகும்
* **Observation space** என்பது நாம் செய்யக்கூடிய கவனிப்புகளின் இடமாகும்


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

சிமுலேஷன் எப்படி செயல்படுகிறது என்பதை பார்க்கலாம். கீழே உள்ள லூப் சிமுலேஷனை இயக்குகிறது, `env.step` முடிவுக்கான கொடி `done` ஐ திருப்பாத வரை. நாம் `env.action_space.sample()` பயன்படுத்தி சீரற்ற செயல்களை தேர்வு செய்வோம், இது பரிசோதனை மிக விரைவாக தோல்வியடையும் வாய்ப்பு உள்ளது (CartPole சூழல் CartPole இன் வேகம், அதன் நிலை அல்லது கோணம் குறிப்பிட்ட வரம்புகளை மீறும்போது முடிவடைகிறது).

> சிமுலேஷன் புதிய சாளரத்தில் திறக்கப்படும். நீங்கள் குறியீட்டை பல முறை இயக்கி அதன் நடத்தை எப்படி இருக்கிறது என்பதை பார்க்கலாம்.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

நீங்கள் கவனிக்கலாம், கண்காணிப்புகளில் 4 எண்கள் உள்ளன. அவை:
- வண்டியின் நிலை
- வண்டியின் வேகம்
- கம்பத்தின் கோணம்
- கம்பத்தின் சுழற்சி வீக்கம்

`rew` என்பது ஒவ்வொரு படியிலும் நாம் பெறும் வெகுமதி. CartPole சூழலில், ஒவ்வொரு சிமுலேஷன் படியிலும் 1 புள்ளி வெகுமதி வழங்கப்படுகிறது, மேலும் CartPole விழாமல் சமநிலையைப் பேணும் நேரத்தை அதிகரிப்பதே இலக்கு.

மீள்பயிற்சி கற்றலில், எங்கள் இலக்கு **policy** $\pi$ ஐ பயிற்சி செய்யும், இது ஒவ்வொரு நிலை $s$ க்கும் எவ்வாறு செயல்பட $a$ என்பதைச் சொல்வது, அடிப்படையாக $a = \pi(s)$ ஆகும்.

நீங்கள் சாத்தியமான தீர்வை விரும்பினால், policy ஐ ஒவ்வொரு செயல்பாட்டிற்கும் சாத்தியக்கூறுகளைத் திருப்பும் ஒன்றாகக் கருதலாம், அதாவது $\pi(a|s)$ என்பது நிலை $s$ இல் செயல்பாடு $a$ ஐ எடுக்க வேண்டிய சாத்தியக்கூறு என்று பொருள்.

## Policy Gradient முறை

எளிய RL ஆல்காரிதத்தில், **Policy Gradient** என அழைக்கப்படும், அடுத்த செயல்பாட்டை கணிக்க ஒரு நரம்பு வலைப்பின்னலை (neural network) பயிற்சி செய்ய உள்ளோம்.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

நாம் பல பரிசோதனைகளை இயக்கி, ஒவ்வொரு இயக்கத்திற்குப் பிறகும் நமது நெட்வொர்க்கை புதுப்பித்து, நெட்வொர்க்கை பயிற்சி செய்யப் போகிறோம். பரிசோதனையை இயக்கி முடிவுகளை (அதாவது **trace**) - அனைத்து நிலைகள், செயல்கள் (மற்றும் அவற்றின் பரிந்துரைக்கப்பட்ட சாத்தியங்கள்), மற்றும் வெகுமதிகளை - திருப்பி அனுப்பும் ஒரு செயல்பாட்டை வரையறுப்போம்:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

நீங்கள் பயிற்சி செய்யப்படாத நெட்வொர்க்குடன் ஒரு எபிசோடைப் இயக்கி, மொத்த பரிசு (எபிசோடின் நீளம் எனவும் அழைக்கப்படுகிறது) மிகவும் குறைவாக இருப்பதை கவனிக்கலாம்:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

கொள்கை சாய்வு الگரிதம் (policy gradient algorithm) தொடர்பான சிக்கலான அம்சங்களில் ஒன்று **தள்ளுபடி செய்யப்பட்ட வெகுமதிகளை** (discounted rewards) பயன்படுத்துவது ஆகும். இதன் கருத்து என்னவென்றால், நாம் விளையாட்டின் ஒவ்வொரு படியிலும் மொத்த வெகுமதிகளின் வெகுமதி வெகுமதிகளை (total rewards) கணக்கிடுகிறோம், மேலும் இந்த செயல்முறையின் போது சில குணகம் $gamma$ பயன்படுத்தி ஆரம்ப வெகுமதிகளை தள்ளுபடி செய்கிறோம். மேலும், நாம் பயிற்சியை பாதிக்க எடை (weight) ஆக பயன்படுத்த உள்ளதால், பெறப்பட்ட வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகுமதி வெகும


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

இப்போது நாங்கள் உண்மையான பயிற்சியை தொடங்குவோம்! நாங்கள் 300 எபிசோடுகளை இயக்குவோம், மேலும் ஒவ்வொரு எபிசோடிலும் கீழே கொடுக்கப்பட்டுள்ளவற்றை செய்வோம்:

1. பரிசோதனையை இயக்கி, அதன் தடத்தை சேகரிக்கவும்
1. எடுத்த நடவடிக்கைகள் மற்றும் கணிக்கப்பட்ட சாத்தியக்கூறுகளுக்கு இடையிலான வேறுபாட்டை (`gradients`) கணக்கிடவும். வேறுபாடு குறைவாக இருக்கும் போது, ​​நாங்கள் சரியான நடவடிக்கையை எடுத்ததற்கான நம்பிக்கை அதிகமாக இருக்கும்.
1. தள்ளுபடி செய்யப்பட்ட வெகுமதிகளை கணக்கிடவும் மற்றும் `gradients`-ஐ தள்ளுபடி செய்யப்பட்ட வெகுமதிகளால் பெருக்கவும் - இது அதிக வெகுமதிகளை கொண்ட படிகள் இறுதி முடிவில் அதிக தாக்கத்தை ஏற்படுத்தும், குறைந்த வெகுமதிகளை கொண்டவற்றை விட.
1. நமது நரம்பியல் வலைப்பின்னலுக்கான எதிர்பார்க்கப்படும் இலக்கு நடவடிக்கைகள் ஓட்டத்தின் போது கணிக்கப்பட்ட சாத்தியக்கூறுகளிலிருந்து ஒரு பகுதியும், கணக்கிடப்பட்ட `gradients`-லிருந்து ஒரு பகுதியும் எடுக்கப்படும். `alpha` அளவுருவை பயன்படுத்தி `gradients` மற்றும் வெகுமதிகள் எந்த அளவுக்கு கணக்கில் கொள்ளப்பட வேண்டும் என்பதை தீர்மானிக்கிறோம் - இதை reinforcement algorithm-இன் *learning rate* என்று அழைக்கப்படுகிறது.
1. இறுதியாக, நாங்கள் நமது வலைப்பின்னலை நிலைகள் மற்றும் எதிர்பார்க்கப்படும் நடவடிக்கைகளில் பயிற்சி செய்து, செயல்முறையை மீண்டும் தொடங்குகிறோம்


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

இப்போது விளைவைக் காண்பதற்காக எபிசோடை ரெண்டரிங் மூலம் இயக்குவோம்:


In [ ]:
_ = run_episode(render=True)

நீங்கள் இப்போது கம்பம் நன்றாக சமநிலைப்படுத்தப்படுவதைப் பார்க்க முடியும் என்று நம்புகிறேன்!

## நடிகர்-விமர்சகர் மாடல்

நடிகர்-விமர்சகர் மாடல் என்பது கொள்கை சாய்வுகளின் மேலும் ஒரு மேம்பாடு ஆகும், இதில் கொள்கை மற்றும் மதிப்பீடு செய்யப்பட்ட வெகுமதிகளை கற்றுக்கொள்வதற்காக நரம்பியல் வலைப்பின்னல்களை உருவாக்குகிறோம். இந்த வலைப்பின்னல் இரண்டு வெளியீடுகளை (அல்லது நீங்கள் அதை இரண்டு தனித்துவமான வலைப்பின்னல்களாகக் காணலாம்) கொண்டிருக்கும்:
* **நடிகர்** கொள்கை சாய்வு மாடலில் போலவே, மாநில சாத்தியக்கூறு விநியோகத்தை வழங்குவதன் மூலம் எடுக்க வேண்டிய நடவடிக்கையை பரிந்துரைக்கும்.
* **விமர்சகர்** அந்த நடவடிக்கைகளிலிருந்து வெகுமதி என்னவாக இருக்கும் என்பதை மதிப்பீடு செய்யும். அது கொடுக்கப்பட்ட மாநிலத்தில் எதிர்காலத்தில் மொத்த மதிப்பீட்டப்பட்ட வெகுமதிகளைத் திருப்பி வழங்கும்.

இவ்வாறு ஒரு மாடலை வரையறுப்போம்:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

நாம் `discounted_rewards` மற்றும் `run_episode` செயல்பாடுகளை சிறிது மாற்ற வேண்டும்:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


இப்போது நாங்கள் முக்கிய பயிற்சி சுற்றத்தை இயக்குவோம். நாங்கள் சரியான இழப்பு செயல்பாடுகளை கணக்கிட்டு, நெட்வொர்க் அளவுருக்களை புதுப்பித்து, கையேடு நெட்வொர்க் பயிற்சி செயல்முறையை பயன்படுத்துவோம்:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## முக்கிய கருத்து

இந்த டெமோவில் இரண்டு RL அல்காரிதங்களை பார்த்தோம்: எளிய கொள்கை சாய்வு மற்றும் மேலும் மேம்பட்ட நடிகர்-விமர்சகர். இந்த அல்காரிதங்கள் நிலை, செயல் மற்றும் வெகுமதி போன்ற சுருக்கமான கருத்துகளுடன் செயல்படுகின்றன - எனவே அவை மிகவும் வேறுபட்ட சூழல்களில் பயன்படுத்தப்படலாம்.

வலியுறுத்தல் கற்றல், இறுதி வெகுமதியைப் பார்த்து மட்டுமே பிரச்சினையைத் தீர்க்க சிறந்த உத்தியை கற்றுக்கொள்ள அனுமதிக்கிறது. லேபிள் செய்யப்பட்ட தரவுத்தொகுப்புகள் தேவையில்லை என்பதால், எங்கள் மாதிரிகளை மேம்படுத்த பல முறை சிமுலேஷன்களை மீண்டும் செய்ய முடிகிறது. இருப்பினும், RL இல் இன்னும் பல சவால்கள் உள்ளன, மேலும் இந்த சுவாரஸ்யமான AI துறையில் அதிக கவனம் செலுத்த முடிவு செய்தால் நீங்கள் அவற்றைப் பற்றி கற்றுக்கொள்ளலாம்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்தவொரு தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
